In [23]:
import utils.ParseXML as ParseXML
import string
import re
import nltk

stopwords = nltk.corpus.stopwords.words('english')

# Specify the path to your XML file
xml_file_path = r'C:\dev\NLP-Sandbox\PURE\requirements-xml\0000 - cctns.xml'
# Define the namespace
namespace = {'ns': 'req_document.xsd'}

# import utils.ParseXML as ParseXML
df = ParseXML.process_xml_with_namespace(xml_file_path, namespace)


In [32]:
import string
import re
import nltk
from nltk.stem import WordNetLemmatizer


stopwords = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()
ps = nltk.PorterStemmer()
lemmatizer = WordNetLemmatizer()


def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

def lemmatizing(tokenized_text):
    # Initialize WordNetLemmatizer
    # Lemmatize each word in the tokenized text
    text = [lemmatizer.lemmatize(word) for word in tokenized_text]
    
    return text


def clean_text_original(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = " ".join([ps.stem(word) for word in tokens if word not in stopwords])
    return text


def clean_text(text, remove_punct=True, remove_stopwords=True):
    """
    Clean the input text by removing punctuation, stopwords, and performing tokenization.
    
    Parameters:
    - text (str): The input text to be cleaned.
    - remove_punct (bool): Flag to indicate whether to remove punctuation or not. Default is True.
    - remove_stopwords (bool): Flag to indicate whether to remove stopwords or not. Default is True.
    
    Returns:
    - cleaned_text (list of str): The cleaned list of tokens.
    """

    # FIXME:  V1.0 tokenized as "v1","0"

    # Remove punctuation
    if remove_punct:
        text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Tokenize the text using regular expression
    tokens = re.findall(r'\b\w+(?:-\w+)*\b', text.lower())
    
    # Remove stopwords if required
    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]
    
    return tokens

def join_list_to_string(attribute):
    if isinstance(attribute, list):
        return ' '.join(attribute)
    else:
        return str(attribute)
    

def clean_and_lemm_text(text):
    text_clean = clean_text(text, False, False)
    text_lemm = lemmatizing(text_clean)
    list_2_word = join_list_to_string(text_lemm)
    
    return list_2_word


In [40]:
df['lemm_text'] = df['text'].apply(lambda x: clean_and_lemm_text(x))
df.head(20)

,tag,text,id,path,cleaned_text,lemm_text
0,title,E-GOVERNANCE MISSION MODE PROJECT (MMP),,req_document/title/title,"[e-governance, mission, mode, project, mmp]",e-governance mission mode project mmp
1,title,CRIME & CRIMINAL TRACKING NETWORK AND SYSTEMS ...,,req_document/title/title,"[crime, criminal, tracking, network, and, syst...",crime criminal tracking network and system cctns
2,title,FUNCTIONAL REQUIREMENTS SPECIFICATION V1.0 (DR...,,req_document/title/title,"[functional, requirements, specification, v1, ...",functional requirement specification v1 0 draft
3,title,MINISTRY OF HOME AFFAIRS GOVERNMENT OF INDIA,,req_document/title/title,"[ministry, of, home, affairs, government, of, ...",ministry of home affair government of india
4,version,1.0,,req_document/version,"[1, 0]",1 0
5,title,INTRODUCTION,1,req_document/p/title,[introduction],introduction
6,title,The Functional Requirements Specifications (FR...,,req_document/p/text_body,"[the, functional, requirements, specifications...",the functional requirement specification fr re...
7,title,FUNCTIONAL OVERVIEW,2,req_document/p/title,"[functional, overview]",functional overview
8,title,CCTNS V1.0 functionality is designed to focus ...,,req_document/p/text_body,"[cctns, v1, 0, functionality, is, designed, to...",cctns v1 0 functionality is designed to focus ...
9,title,DESCRIPTION OF THE MODULES AND FUNCTIONAL REQU...,3,req_document/p/title,"[description, of, the, modules, and, functiona...",description of the module and functional requi...


In [41]:
import utils.CleanData as CleanData
from sklearn.feature_extraction.text import CountVectorizer

# Analysier paramether we can pass a function to clean the text
count_vect = CountVectorizer(analyzer=clean_and_lemm_text)
X_counts = count_vect.fit_transform(df['lemm_text'])

# This will output ("how many rows where analysied", "How many unique words there are")
print(X_counts.shape)
print(count_vect.get_feature_names_out())

# FIXME: The clean_and_lemm_text is not returnig a string, but a list.


(154, 38)
[' ' '-' '0' '1' '2' '3' '4' '5' '6' '7' '8' '9' 'a' 'b' 'c' 'd' 'e' 'f'
 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x'
 'y' 'z']
